# SDL.3 Building Asset Trees

### Objective: Learn how to build an asset tree using a CSV file and modify the asset tree with calculations and roll-up statistics.  Push the completed asset tree to the Seeq Server.

### Scenario: You have developed an algorithm to calculate the health of a Cooling Tower based on temperature data within each area. In the final step, you would like to organize the data in an Asset Tree, so the team has  access to the raw and calculated data for monitoring.

<div class="alert alert-block alert-warning">
<b>Discussion Topic:</b> What is an Asset Tree?
        <details>
    <summary>✼</summary>
            
### Asset Trees Introduction

Asset trees are a foundational tool that can be used to wrangle the full analytic capabilities of Seeq's software. They sort physical locations, pieces of equipment, and data on that equipment into a hierarchical structure. Organizing your data into an asset tree allows you to:
- Utilize asset swapping to rapidly create identical visualizations for different pieces of equipment
- Write high-value calculations for your components and scale them across all components in your tree
- Automatically generate scalable content and custom analyses
- Use your tree as a starting point for roll-ups, calculations, displays, dashboards, and reports
            
</details>
</div>


## Step 0: Defining an Asset Tree

There are multiple ways to define an asset tree in SPy using the: `spy.assets.Tree()`.

<div class="alert alert-block alert-success">
<b>Instructor Note:</b> Review the three ways of <a href ="https://python-docs.seeq.com/user_guide/spy.assets/Asset%20Trees%201%20-%20Introduction.html#defining-an-asset-tree">Defining an Asset Tree</a>    
</div>




Click the link for more information on [spy.assets](https://python-docs.seeq.com/user_guide/spy.assets/index.html)


## Step 1: Build a CSV File
In this course, we will start by creating a tree with a custom structure defined by a CSV (comma-separated values) file. When this file is given as input to `spy.assets.Tree`, SPy will look at each row in the file, find an item in Seeq corresponding to the row, and place it into a newly created asset tree at the specified location.

The following columns are required in your CSV file.  
- **Name** tells SPy what item to pull from the Seeq Server.
- **Level** tells SPy where in the tree to put the item.
  - If a level column is empty for a particular row, then the rows above are referred to. For example, the third row above has levels *My CSV Tree*, *Cooling Tower 1*, and *Area B*.
- **Friendly Name** tells SPy what to call the item after putting it in the tree.
- **ID** <i>(optional)</i> column can help SPy find items from Seeq Server that don't have unique names.

| Name                     | Level 1     | Level 2         | Level 3 | Friendly Name     |
|--------------------------|-------------|-----------------|---------|-------------------|
| Area A_Temperature | My CSV Tree | Cooling Tower 1 | Area A  | Temperature       |
| Area B_Temperature |             |                 | Area B  | Temperature       |
| Area C_Temperature |             |                 | Area C  | Temperature       |
| Area D_Temperature |             | Cooling Tower 2 | Area D  | Temperature      |
| Area E_Temperature |             |                 | Area E  | Temperature      |



In [ ]:
my_csv_tree = spy.assets.Tree('spy_tree_example_r6.csv')
my_csv_tree.visualize()

<div class="alert alert-block alert-warning">
<b>Discussion Topic:</b> When would you want to use the ID column instead of the Name column?
</div>


Let's push the asset tree to Seeq and explore how it looks in Workbench.

In [ ]:
my_csv_tree.push()

<div class="alert alert-block alert-warning">
<b>Discussion Topic:</b> Where is this asset tree scoped and how can we change it?
        <details>
    <summary>✼</summary>
<i>The asset tree is scoped to <tt>workbook='Data Lab >> Data Lab Analysis' </tt> (ref Function Documentation). </i>
</details>
</div>

**Optional Exercise:** Scope the asset tree to a new location to demonstrate how scoping affects the location of the tree. `workbook='Data Lab >> New Location'`

***

## Step 2:  Insert a Calculation

The next step to building your asset tree is to add more items to it using the `.insert()` function.


<div class="alert alert-block alert-success">
<b>Instructor Note:</b> Review the three ways of <a href ="https://python-docs.seeq.com/user_guide/spy.assets/Asset%20Trees%201%20-%20Introduction.html#inserting-items-into-the-tree"> Inserting Items into the Tree</a>    
</div>



Let’s start by inserting a calculation. A calculation requires a name, a formula, and a collection of formula parameters. The formula is written in [Seeq Formula Language](https://support.seeq.com/space/KB/143884328/Formula), and the formula parameters assign variables in the formula to items in your tree.

As we mentioned in SDL.1, you can apply formulas at scale.  Let’s apply the formula provided by the engineers from SDL.1.

`'$signal.remove((($signal < 0).merge(15min))).agilefilter(5min)'`

<div class="alert alert-block alert-info">
    <b>Tip:</b> To show the docstring in pager mode press <tt>shift + tab</tt> four times.
</div>

In [ ]:
my_csv_tree.insert(name='Temperature Cleansed',
               formula='$signal.remove((($signal < 0).merge(15min))).agilefilter(5min)',
               formula_parameters={'$signal': 'Temperature'},
               parent='Area ?')
my_csv_tree.visualize()

***

## Step 3: Insert a Condition
Similarly, we can add conditions into the asset tree. 

In [ ]:
my_csv_tree.insert(name='Too Hot',
               formula='$signal > 100',
               formula_parameters={'$signal': 'Temperature Cleansed'},
               parent='Area ?')
my_csv_tree.visualize()

***

## Step 4: Insert a Roll-up Calculation

**Roll-up calculations** are a great way to evaluate summary statistics across multiple assets in your tree in order to monitor the health and performance of your assets. To insert a roll-up calculation, use the `roll_up_statistic` and `roll_up_parameters` inputs to the `insert()` function.

In [ ]:
my_csv_tree.insert(name='Average Temperature of All Areas',
                   roll_up_statistic='Average',
                   roll_up_parameters='Area ? >> Temperature Cleansed',
                   parent='Cooling Tower ?')
my_csv_tree

***

## Step 5: Insert a Signal
Let's add in the Health Score calculated from SDL.2 into our tree. First, let's search for the signal, and then insert it into our tree. 

<div class="alert alert-block alert-success">
<b>Instructor Note:</b> If you didn't complete SDL.2, Step 5 can be skipped.
</div>

In [ ]:
search_results = spy.search(query = {'Name': 'CT1 Health Score'})

search_results

In [ ]:
my_csv_tree.insert(children=search_results,
               friendly_name='CT1 Health Score',
               parent='Cooling Tower 1')

my_csv_tree.visualize()

Perform the same steps for Cooling Tower 2

In [ ]:
search_results = spy.search(query = {'Name': 'CT2 Health Score'})

my_csv_tree.insert(children=search_results,
               friendly_name='CT2 Health Score',
               parent='Cooling Tower 2')

my_csv_tree.visualize()

<div class="alert alert-block alert-warning">
<b>Discussion Topic:</b> When should you use Asset Groups in Seeq Workbench, vs asset trees in Data Lab? 
            <details>
    <summary>✼</summary>
<i> When thinking about building assets, you want to consider the number of assets you are working with and how many levels you want to create. Asset groups can only create a single level tree; however they require no code and can leverage existing asset trees.  Asset trees can be scaled to 1000s of assets and be multi level. </i>
</details>
</div>
    

***

## Step 6: Pushing a Tree

The last step to working with an asset tree in SPy is to push your changes to Seeq! 

In [ ]:
my_csv_tree.push()

***

<div class="alert alert-block alert-warning">
<b>Discussion Topic:</b> Can you think about opportunities to use assets trees on something other than assets? 
     <details>
    <summary>✼</summary>
         <i>Assets can be used to analyse individual steps of a batch process or even to scale golden profiles. </i>    </details>
</div>



## SDL.3 Summary

The steps above have been combined into one cell. 

In [ ]:
#Step 1: Build a CSV File
my_csv_tree = spy.assets.Tree('spy_tree_example_r6.csv', workbook='Data Lab Training >> Data Lab Training')

#Step 2: Insert a Calculation
my_csv_tree.insert(name='Temperature Cleansed',
               formula='$signal.removeoutliers()',
               formula_parameters={'$signal': 'Temperature'},
               parent='Area *')


#Step 3: Insert a Condition
my_csv_tree.insert(name='Too Hot',
               formula='$signal > 100',
               formula_parameters={'$signal': 'Temperature'},
               parent='Area *')


#Step 4: Insert a Roll Up Calculation
my_csv_tree.insert(name='Average Temperature of All Areas',
                   roll_up_statistic='Average',
                   roll_up_parameters='Area ? >> Temperature Cleansed',
                   parent='Cooling Tower ?')

#Step 5: Insert a Signal
search_results = spy.search({'Name': 'CT1 Health Score',
                             'Type': 'StoredSignal',
                             'Datasource Name': 'Seeq Data Lab'
                             },workbook ='Data Lab Training >> Workbook Test_D')
my_csv_tree.insert(children=search_results,
               friendly_name='CT1 Health Score',
               parent='Cooling Tower 1')

#Step 6: 
my_csv_tree.push()

***

## Step 7. Other Operations on an  Asset Tree (Optional)

In the steps above, we inserted calculations (Temperature Cleansed, Too Hot, Roll Up Calculations) and signals (Health Score).

In this optional exercise, we will add Cooling Tower 3 and Area G, Area H, Area I. 


In [ ]:
my_csv_tree.insert(parent='My CSV Tree',children='Cooling Tower 3')
my_csv_tree.visualize()

In [ ]:
my_csv_tree.insert(parent='Cooling Tower 3',children=['Area G','Area H','Area I'])
my_csv_tree.visualize()

In [ ]:
my_csv_tree.remove('Area I')
my_csv_tree.visualize()

In [ ]:
my_csv_tree.push()